<a href="https://colab.research.google.com/github/fjadidi2001/DataScienceJourney/blob/master/PINN_1D_Poisson_equation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

> Physics-Informed Neural Networks (PINNs) are a type of neural network that leverages both data and physical laws (represented by differential equations) to model complex systems. They are particularly useful for solving partial differential equations (PDEs), where traditional numerical methods may be computationally expensive or infeasible. PINNs incorporate these physical laws as part of the loss function, allowing the network to not only fit data but also adhere to the governing physics.

# Steps to Solve the Poisson Equation Using PINNs
1. Define the Neural Network: A fully connected neural network is used to approximate the solution 𝑢(𝑥,𝑦) of the Poisson equation.
2. Formulate the PDE Residual: The residual of the Poisson equation is derived by substituting the network's output into the equation.
3. Define the Loss Function: The loss function consists of two parts:
- PDE Loss: Ensures that the network's output satisfies the Poisson equation.
- Boundary Condition Loss: Ensures that the network satisfies the boundary conditions.
4. Train the Network: The network is trained by minimizing the combined loss.


In [1]:
import tensorflow as tf
import numpy as np

# Step 1: Define the neural network model
class PINN(tf.keras.Model):
    def __init__(self):
        super(PINN, self).__init__()
        self.dense1 = tf.keras.layers.Dense(20, activation='tanh')
        self.dense2 = tf.keras.layers.Dense(20, activation='tanh')
        self.dense3 = tf.keras.layers.Dense(20, activation='tanh')
        self.dense4 = tf.keras.layers.Dense(1, activation=None)

    def call(self, X):
        x = self.dense1(X)
        x = self.dense2(x)
        x = self.dense3(x)
        return self.dense4(x)

# Step 2: Define the physics-informed loss function
def pde_loss(model, X):
    with tf.GradientTape(persistent=True) as tape:
        tape.watch(X)
        u = model(X)
        u_x = tape.gradient(u, X)[:, 0:1]
        u_y = tape.gradient(u, X)[:, 1:2]
        u_xx = tape.gradient(u_x, X)[:, 0:1]
        u_yy = tape.gradient(u_y, X)[:, 1:2]
    del tape
    # Poisson equation residual (Δu = f(x, y))
    residual = u_xx + u_yy + 1.0  # assuming f(x, y) = -1
    return tf.reduce_mean(tf.square(residual))

# Step 3: Define the boundary condition loss
def boundary_loss(model, X_boundary):
    u_boundary = model(X_boundary)
    return tf.reduce_mean(tf.square(u_boundary))  # u = 0 on boundary

# Step 4: Total loss function
def total_loss(model, X, X_boundary):
    return pde_loss(model, X) + boundary_loss(model, X_boundary)

# Step 5: Generate training data (collocation points and boundary points)
def generate_training_data(N_collocation, N_boundary):
    # Collocation points inside the domain
    X_collocation = np.random.rand(N_collocation, 2)
    # Boundary points
    X_boundary = np.vstack([
        np.random.rand(N_boundary, 1), np.zeros((N_boundary, 1)),
        np.zeros((N_boundary, 1)), np.random.rand(N_boundary, 1),
        np.ones((N_boundary, 1)), np.random.rand(N_boundary, 1),
        np.random.rand(N_boundary, 1), np.ones((N_boundary, 1))
    ]).T
    return X_collocation, X_boundary

# Step 6: Train the PINN
def train_pinn(model, X, X_boundary, epochs, learning_rate):
    optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)
    for epoch in range(epochs):
        with tf.GradientTape() as tape:
            loss_value = total_loss(model, X, X_boundary)
        grads = tape.gradient(loss_value, model.trainable_variables)
        optimizer.apply_gradients(zip(grads, model.trainable_variables))

        if epoch % 100 == 0:
            print(f'Epoch {epoch}, Loss: {loss_value.numpy()}')

# Step 7: Visualize the solution
def plot_solution(model):
    x = np.linspace(0, 1, 100)
    y = np.linspace(0, 1, 100)
    X, Y = np.meshgrid(x, y)
    X_flat = X.flatten().reshape(-1, 1)
    Y_flat = Y.flatten().reshape(-1, 1)
    U_pred = model(np.hstack([X_flat, Y_flat])).numpy().reshape(100, 100)

    plt.contourf(X, Y, U_pred, 20, cmap='viridis')
    plt.colorbar()
    plt.title('PINN Solution of the Poisson Equation')
    plt.xlabel('x')
    plt.ylabel('y')
    plt.show()

# Main script
N_collocation = 10000  # Number of collocation points
N_boundary = 400       # Number of boundary points
epochs = 5000          # Number of training epochs
learning_rate = 0.001  # Learning rate

X_collocation, X_boundary = generate_training_data(N_collocation, N_boundary)
model = PINN()
train_pinn(model, X_collocation, X_boundary, epochs, learning_rate)
plot_solution(model)


ValueError: Passed in object [[0.07733232 0.94513172]
 [0.37562977 0.04169731]
 [0.84557048 0.92807062]
 ...
 [0.80995805 0.86451098]
 [0.31313663 0.91349819]
 [0.68428918 0.92389123]] of type 'ndarray', not tf.Tensor or tf.Variable or ExtensionType.